## 1. Import Relevant Libraries and Reading of Data

* 1a. Importing Relevant Libraries

In [50]:
# Import Required Libaries
import keras
import keras.backend as K
from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, TimeDistributed

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing

from keras.layers import Bidirectional

import math 

* 1b. Reading in the Data:
-> Training Data
-> Testing Data
-> True Labels (RUL) for the Testing Data

In [51]:
# Reading in the data (txt files)

# Train Data
train_df = pd.read_csv("/content/train_FD002.txt", sep=" ", header=None)
# Test Data
test_df = pd.read_csv("/content/test_FD002.txt", sep=" ", header=None)
# True RUL Data for the Test Set
truth_df = pd.read_csv("/content/RUL_FD002.txt", sep=" ", header=None)

* 1c. We set a seed for Reproducibility of Results

In [52]:
np.random.seed(1234)  
PYTHONHASHSEED = 0

* 1d. Create a path to save the Deep Learning model output 

In [53]:
# define a path to save model
model_path = '/content/Output/regression_model.h5'

## 2. Data Preprocessing
- Drop the last 2 columns which consists of all null values
- Rename columns according to the Engine ID, Cycle, the 3 respective Operating Settings and the 21 Operating Sensors.
- Derive the RUL from the train data. This is because the RUL for the train data is not explicity provided.
- MinMax Normalisation to transform our features' values to a value between 0 and 1; 0 represents the minimum output value of the sensor value, while 1 represents the maximum output value of the sensor value.
- Clipping the upper limit of the RUL of aircrafts to mimic a more accurate degradation pattern of the aircraft engine with increasing usage.
- Transform the data into a form (3-Dimensional Form) that can be fed into the deep learning model. 

* 2a. Drop the last 2 columns for Training, Testing and True Labels Dataframes

In [54]:
# Drop the last 2 columns for Train, Test and True RUL Dataframes
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

* 2b. Rename Columns in this form: 
-> Engine ID
-> 3 Operatinh Settings
-> 21 Sensors

In [55]:
# Rename columns into readable forms, namely, by: 
# [Engine ID, Operational Settings, Sensors]


train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
train_df = train_df.sort_values(['id','cycle'])


test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']


* 2c. Derive RUL for the Training Data

In [56]:
# Derive the RUL for the Train Data
def extract_rul(data, factor = 0):

    # Get the total number of cycles for each unit, i.e. Each Engine ID
    rul = pd.DataFrame(data.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']

    # Merge the maximum cycle into the original dataframe
    data = data.merge(rul, on=['id'], how='left')

    # Actual calculation of RUL
    data['RUL'] = data['max'] - data['cycle']

    # Drop the 'max' column, which is now redundant
    data.drop(columns=['max'], axis = 1, inplace = True)
  
    return data[data['cycle'] > factor]

# Apply the function on the training data
train_df = extract_rul(train_df, factor = 0)

* 2d. Clipping the maximum RUL to aircraft engines for Training Data 
> This is to accurately mimic the degradation of aircraft engines after a certain period of usage

In [57]:
# Clipping the maximum RUL to aircraft engines (for train data)
train_df['RUL'] = train_df['RUL'].clip(upper=125)

* 2e. MinMax Normalisation of Training Data

In [58]:
# MinMax normalization of Operational Settings and Sensor Values (from 0 to 1) for train set
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)

# MinMax normalization of Operational Settings and Sensor Values (from 0 to 1) for test set
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)





In [59]:
# Generate labels (the RUL) for the Test Data using the dataset containing the true RUL for the Test Data.
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
# rename column in Truth Dataframe
truth_df.columns = ['rul_init']
# assign the RUL to the Engine IDs by adding a column in Truth Dataframe
truth_df['id'] = truth_df.index + 1
# assign RUL to a newly-named column, max
truth_df['max'] = rul['max'] + truth_df['rul_init']
truth_df.drop('rul_init', axis=1, inplace=True)

# Merge Truth Dataframe with Test Dataframe to put the actual RUL together with the data in Test Dataframe
# Merge based on Engine ID
test_df = test_df.merge(truth_df, on=['id'], how='left')
# Final RUL matched to Engine ID
test_df['RUL'] = test_df['max'] - test_df['cycle']

# Clipping the maximum RUL to aircraft engines.
test_df['RUL'] = test_df['RUL'].clip(upper=125)
test_df.drop('max', axis=1, inplace=True)

In [60]:
# Transforming data into a form that can be fed in to the Deep Learning Model 
# The Form is a(3 Dimensional Form): (samples, time steps, features) 

# Assign sequence length of 50 (why?)
sequence_length = 50
def gen_sequence(id_df, seq_length, seq_cols):
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]

In [61]:
# Feature columns consists of the Operational Settings and Sensor Columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

# Generate a sequence with the gen_sequence function to get the 3-Dimensional Form 
# Convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)

In [62]:
# Generate the RUL labels 
def gen_labels(id_df, seq_length, label):
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    return data_matrix[seq_length:num_elements, :]

# Generate the labels using the gen_labels function
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) 
             for id in train_df['id'].unique()]
# Convert to numpy array
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(40759, 1)

## 3. Model Building

In [63]:
# Defining R2 to be used as an evaluation metric in the Deep Learning Model Evaluation Metrics
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

## Deep Learning Architecture and Topology

In [64]:
# Building the Deep Learning Architecture

# Defining the features to be fed into the input of the layers in the Deep Learning Model
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

# Sequential Model
model = Sequential()
# Add a Bi-LSTM Layer
model.add(Bidirectional(LSTM(20, return_sequences=True), input_shape=(sequence_length, nb_features)))
# Add a Dropout Layer after the Bi-LSTM Layer to minimize overfitting  
model.add(Dropout(0.2))
# Add a LSTM Layer
model.add(LSTM(units=50,return_sequences=False, activation = 'tanh'))
# Add a Dropout Layer after the LSTM Layer to minimize overfitting  
model.add(Dropout(0.2))
# Dense Layer
model.add(Dense(units=nb_out))
# Activation Function 
model.add(Activation("linear"))
# Compile model, set metrics for evaluation
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mse',r2_keras])
# For model observation
print(model.summary())

# Fit the Deep Learning network on our training data
history = model.fit(seq_array, label_array, epochs=5, batch_size=10, validation_split=0.05, verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)])


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 50, 40)            7360      
_________________________________________________________________
dropout_4 (Dropout)          (None, 50, 40)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                18200     
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 25,611
Trainable params: 25,611
Non-trainable params: 0
__________________________________________________

In [65]:
# Metrics from fitting the model on our Training Data
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('\nMSE: {}'.format(scores[1]))
print('\nR^2: {}'.format(scores[2]))

204/204 [==============================] - 1s 6ms/step - loss: 498.8837 - mse: 498.8837 - r2_keras: 0.6191

MSE: 498.88372802734375

R^2: 0.6191354393959045


In [66]:
# Adjusted R2 score
def adjusted_r2(r2, p, n):
  numerator = (1-r2)*(n-1)
  denom = n - p - 1
  result = 1 - (numerator/denom)
  return result

print('Adjusted R2:')
print(adjusted_r2(scores[2], nb_features, len(label_array)))



Adjusted R2:
0.6189016826381135


In [67]:
# Preparing Test Data to be fed into our model for evaluation
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)

In [68]:
# Preparing Test Data to be fed into our model for evaluation
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]
label_array_test_last = test_df.groupby('id')['RUL'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)

In [69]:
# Test Data Metrics after running our model
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose = 1, batch_size = 200)
print('\nMSE: {}'.format(scores_test[1]))
print('\nR^2: {}'.format(scores_test[2]))
print('\nRMSE:')
print(math.sqrt(scores_test[1]))

2/2 [==============================] - 0s 8ms/step - loss: 388.4691 - mse: 388.4691 - r2_keras: 0.7658

MSE: 388.4690856933594

R^2: 0.7658010125160217

RMSE:
19.709619115887534


In [70]:
# To obtain our model's prediction on the Test Data
scores_test = model.predict(seq_array_test_last)
scores_test


array([[  6.7662206],
       [116.90077  ],
       [ 97.342735 ],
       [107.206276 ],
       [ 15.140278 ],
       [114.28436  ],
       [ 11.647469 ],
       [ 38.184948 ],
       [ 11.585176 ],
       [ 98.84564  ],
       [ 24.801308 ],
       [114.18645  ],
       [ 45.5145   ],
       [  8.982037 ],
       [ 39.126545 ],
       [103.864525 ],
       [ 90.57275  ],
       [ 31.114506 ],
       [ 94.58545  ],
       [ 53.60908  ],
       [ 18.05445  ],
       [ 73.65123  ],
       [ 85.2809   ],
       [110.72077  ],
       [116.268875 ],
       [104.1035   ],
       [ 82.9159   ],
       [103.01494  ],
       [ 15.835787 ],
       [ 55.29033  ],
       [116.35204  ],
       [112.56723  ],
       [ 81.361374 ],
       [ 48.412544 ],
       [ 60.772785 ],
       [ 51.31762  ],
       [ 27.467094 ],
       [ 47.50279  ],
       [ 44.66049  ],
       [ 30.419733 ],
       [ 11.280405 ],
       [ 94.93619  ],
       [113.30233  ],
       [ 36.202633 ],
       [113.11177  ],
       [ 1

In [71]:
test_df.id.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [72]:
from sklearn.metrics import mean_squared_error
rms = mean_squared_error(label_array_test_last, scores_test, squared=False)
rms

19.70962

In [73]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print(r2_score(label_array_test_last, scores_test))

0.7827949367023176
